Database Import

In [20]:
import pandas as pd
#main_df = pd.read_csv("../data/output/main_df28.csv")
#main_df = pd.read_csv("../data/output/main_df29.csv")
main_df = pd.read_csv("../data/output/main_df30.csv")

Data management - Average Observations Methodology

In [21]:
import pandas as pd
import numpy as np

# Assume main_df is already in memory
# main_df = pd.read_csv('path_to_your_csv_file/main_df12.csv')

# Define a list of common non-standard NaN indicators
nan_indicators = ['nan', 'NaN', 'NAN', '..', '...', '-', '']

# Replace these indicators with standard NaN
main_df.replace(nan_indicators, np.nan, inplace=True)

# Apply float conversion for specific columns, handling NA values
main_df["control_corrupt"] = main_df["control_corrupt"].apply(lambda x: float(x) if pd.notna(x) else np.nan)
main_df["CO2 emissions from transport (% of total fuel combustion)"] = main_df["CO2 emissions from transport (% of total fuel combustion)"].apply(lambda x: float(x) if pd.notna(x) else np.nan)
main_df["Electric power transmission and distribution losses (% of output)"] = main_df["Electric power transmission and distribution losses (% of output)"].apply(lambda x: float(x) if pd.notna(x) else np.nan)
main_df["Internally displaced persons, new displacement associated with disasters (number of cases)"] = main_df["Internally displaced persons, new displacement associated with disasters (number of cases)"] / (main_df["population"]*1000000)

def calculate_average_last_10(df, country_col='country', year_col='year', code_col='country_code'):
    # Initialize a list to hold the results
    results = []
    
    # Group by country
    for country, group in df.groupby(country_col):
        # Sort by year in descending order
        group_sorted = group.sort_values(by=year_col, ascending=False)
        
        # Initialize dictionaries to store mean values and corresponding years
        mean_values = {}
        years_info = {}
        
        # For each numeric column, calculate the mean of the last 10 non-NA values
        for col in group_sorted.select_dtypes(include='number').columns:
            last_10_non_na = group_sorted[col].dropna().head(10)
            mean_values[col] = last_10_non_na.mean()
            
            # Capture the corresponding years for these last 10 non-NA values
            years_taken = group_sorted[year_col][group_sorted[col].notna()].head(10)
            years_info[col + '_years'] = ', '.join(years_taken.astype(str).tolist())
        
        # Add the country and code to the mean values dictionary
        mean_values[country_col] = country
        mean_values[code_col] = group[code_col].iloc[0]
        
        # Combine mean values and years info into one dictionary
        mean_values.update(years_info)
        
        # Append the result to the results list
        results.append(mean_values)

    # Create a DataFrame from the results
    averaged_data = pd.DataFrame(results)
    
    return averaged_data

# Function to calculate last observation
def calculate_last_observation(df, country_col='country', year_col='year', code_col='country_code'):
    # Initialize a list to hold the results
    results = []
    
    # Group by country
    for country, group in df.groupby(country_col):
        # Sort by year in descending order
        group_sorted = group.sort_values(by=year_col, ascending=False)
        
        # Initialize dictionaries to store last values and corresponding years
        last_values = {}
        
        # For each numeric column, take the last non-NA value
        for col in group_sorted.select_dtypes(include='number').columns:
            last_non_na = group_sorted[col].dropna().head(1)
            last_values[col] = last_non_na.iloc[0] if not last_non_na.empty else np.nan
            
            # Capture the corresponding year for this last non-NA value
            years_taken = group_sorted[year_col][group_sorted[col].notna()].head(1)
            last_values[col + '_last'] = years_taken.iloc[0] if not years_taken.empty else np.nan
        
        # Add the country and code to the last values dictionary
        last_values[country_col] = country
        last_values[code_col] = group[code_col].iloc[0]
        
        # Append the result to the results list
        results.append(last_values)

    # Create a DataFrame from the results
    last_observation_data = pd.DataFrame(results)
    
    return last_observation_data

# Apply the function to calculate the averages
main_cs2 = calculate_average_last_10(main_df, country_col='country', year_col='year', code_col='country_code')
main_cs2["year"] = 2023

# Apply the function to calculate the last observation
main_cs_last = calculate_last_observation(main_df, country_col='country', year_col='year', code_col='country_code')

# Clean up auxiliary variables
del nan_indicators, calculate_average_last_10, calculate_last_observation

# Split the dataframes based on column names
main_cs_years = main_cs2.loc[:, main_cs2.columns.str.endswith('_years') | main_cs2.columns.isin(['country', 'country_code', 'year'])]
main_cs_years_last = main_cs_last.loc[:, main_cs_last.columns.str.endswith('_last') | main_cs_last.columns.isin(['country', 'country_code', 'year'])]

main_cs = main_cs2.loc[:, ~main_cs2.columns.str.endswith('_years') | main_cs2.columns.isin(['country', 'country_code', 'year'])]
main_cs_last = main_cs_last.loc[:, ~main_cs_last.columns.str.endswith('_last') | main_cs_last.columns.isin(['country', 'country_code', 'year'])]

# Now you have three dataframes:
# 1. main_cs: Dataframe with the average of the last 10 observations
# 2. main_cs_years: Dataframe with the years corresponding to the last 10 observations used for averaging
# 3. main_cs_last: Dataframe with the last available observation and the corresponding year stored in _last columns

In [22]:
variable_names = {
    'index': 'index',
    'country': 'country',
    'country_code': 'country_code',
    'year': 'year',
    'Central government debt, total (% of GDP)': 'gov_debt_gdp',
    'Domestic credit to private sector (% of GDP)': 'priv_cred_gdp',
    'Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)': 'edu_att_uppr',
    'GDP per person employed (constant 2021 PPP $)': 'gdp_per_empl',
    'Intentional homicides (per 100,000 people)': 'intent_hom',
    'Interest rate spread (lending rate minus deposit rate, %)': 'int_rate_sprd',
    'Internally displaced persons, new displacement associated with disasters (number of cases)': 'displ_pax_dis',
    'Logistics performance index: Ability to track and trace consignments, score (1=low to 5=high)': 'log_track',
    'Logistics performance index: Competence and quality of logistics services, score (1=low to 5=high)': 'log_qual',
    'Logistics performance index: Ease of arranging competitively priced shipments (1=low to 5=high)': 'log_ease',
    'Logistics performance index: Efficiency of the clearance process, score (1=low to 5=high)': 'log_clear',
    'Logistics performance index: Frequency with which shipments reach consignee within scheduled or expected time, score (1=low to 5=high)': 'log_freq',
    'Logistics performance index: Quality of trade and transport-related infrastructure (1=low to 5=high)': 'log_infra',
    'People using at least basic drinking water services (% of population)': 'basic_drink',
    'People using at least basic sanitation services (% of population)': 'basic_san',
    'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)': 'pov_hc_215',
    'Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population)': 'pov_hc_365',
    'Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population)': 'pov_hc_685',
    'Public private partnerships investment in transport (current US$)': 'ppp_transp',
    'Renewable energy consumption (% of total final energy consumption)': 'renew_enrgy',
    'Research and development expenditure (% of GDP)': 'rnd_exp_gdp',
    'Researchers in R&D (per million people)': 'resrch_per_m',
    'School enrollment, preprimary (% gross)': 'preprim_enrl',
    'School enrollment, primary (% net)': 'prim_enrl',
    'School enrollment, secondary (% net)': 'sec_enrl',
    'Access to electricity (% of rural population with access': 'rural_ele',
    'Access to electricity (% of urban population with access': 'urban_ele',
    'Age-standardized suicide rates (per 100 000 population)': 'suicide_rate',
    'Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%)': 'health_exp',
    'Deaths per 1 000 live births': 'deaths_birth',
    'Maternal mortality ratio (per 100 000 live births)': 'maternal_mort',
    'Medical doctors (per 10,000)': 'med_docs_10k',
    'Premature deaths due to noncommunicable diseases (NCD) as a proportion of all NCD deaths': 'prem_deaths',
    'UHC Service Coverage sub-index on noncommunicable diseases': 'uhc_ncd_cov',
    'Account, female (% age 15+)': 'acct_female',
    'Account, male (% age 15+)': 'acct_male',
    'Borrowed from a formal financial institution (% age 15+)': 'borrow_fin',
    'Borrowed from a formal financial institution, female (% age 15+)': 'borrow_fem',
    'Borrowed from a formal financial institution, male (% age 15+)': 'borrow_male',
    'Financial institution account (% age 15+)': 'fin_acct',
    'Made or received a digital payment (% age 15+)': 'dig_pay',
    'Account, GPI (% age 15+)': 'acct_gpi',
    'Borrowed from a formal financial institution, GPI (% age 15+)': 'borrow_gpi',
    'Agriculture': 'agriculture',
    'Air Pollution': 'air_pollut',
    'Air Quality': 'air_qual',
    'Biodiversity & Habitat': 'bio_hab',
    'Climate Change Mitigation': 'clim_chng_m',
    'Fisheries': 'fisheries',
    'Forests': 'forests',
    'Heavy Metals': 'heavy_met',
    'Sanitation & Drinking Water': 'san_drink',
    'Waste Management': 'waste_mgmt',
    'Water Resources': 'water_res',
    'Completion rate, primary education, both sexes (%)': 'prim_compl',
    'Completion rate, upper secondary education, both sexes (%)': 'upper_sec_c',
    'Completion rate, upper secondary education, GDI (%)': 'upper_sec_g',
    'Proportion of students at the end of lower secondary education achieving at least a minimum proficiency level in mathematics, both sexes (%)': 'prof_math',
    'Proportion of students at the end of lower secondary education achieving at least a minimum proficiency level in reading, both sexes (%)':'prof_read',
    'Net enrolment rate, early childhood education, both sexes (%)': 'early_enrl',
    'Percentage of pupils enrolled in primary education who are at least 2 years over-age for their current grade, both sexes (%)': 'ovrage_prim',
    'Percentage of pupils enrolled in lower secondary general education who are at least 2 years over-age for their current grade, both sexes (%)': 'ovrage_sec',
    'Participation rate of youth and adults in formal and non-formal education and training in the previous 12 months, 15-24 years old, both sexes (%)': 'part_formal',
    'Gross enrolment ratio, primary and secondary, female (%)': 'gr_enrl_fem',
    'Gross enrolment ratio, primary and secondary, male (%)': 'gr_enrl_male',
    'Gross enrolment ratio, primary and secondary, GPI (%)': 'gr_enrl_gpi',
    'Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 15+)': 'women_viol',
    'Proportion of women subjected to physical and/or sexual violence in the last 12 mon ths (modeled estimate, % of ever partnered women ages 15+)': 'viol_12mon',
    'Network Readiness Index: subpillar: Bussineses': 'net_read_biz',
    'Population using safely managed sanitation services (%)': 'safe_san',
    'Agricultural water withdrawal as % of total renewable water resources': 'agri_water',
    'Agricultural R&D spending (constant 2011 PPP$)': 'agri_rnd',
    'Agricultural researchers (FTE)': 'agri_resrch',
    'Total Agro Emissions': 'agro_emiss',
    'rtfpna': 'rtfpna',
    'rgdpna': 'rgdpna',
    'pop': 'pop',
    'population': 'population',
    'gdp_ppp_pc': 'gdppcppp_const',
    'gdppcppp_const': 'gdp_ppp',
    'Electric power transmission and distribution losses (% of output)': 'elec_losses',
    'CO2 emissions from transport (% of total fuel combustion)': 'co2_trans',
    'Number_of_products_Value': 'prod_value',
    'Concentration_Index_Value': 'concen_idx',
    'egov_index': 'egov_idx',
    'egov_index2': 'egov_idx2',
    'pr_pop_rate': 'prison_rate',
    'water_use_ef': 'water_eff',
    'control_corrupt': 'ctrl_corrpt',
    'eci_rank': 'eci_rank',
    'Ratio of female to male labor force participation rate (%)': 'lf_part_rat',
    ' Percent of children living in monetary poor households in 2022 $2.15':'child_p_215',
    ' Percent of children living in monetary poor households in 2022  $3.65':'child_pov_365',
    ' Percent of children living in monetary poor households in 2022  $6.85':'child_pov_685',
    "GCI 4.0: Distortive effect on competition of taxes and subsidies (1-7)": "comp_tax_sub",
    "GCI 4.0: Burden of government regulation (1-7)": "govt_reg_bur",
    "GCI 4.0: Extent of market dominance (1-7)": "mkt_dominance",
    "GCI 4.0: Hiring and firing practices (1-7)": "hire_fire",
    "GCI 4.0: Flexibility of wage determination (1-7)": "wage_flex",
    "GCI 4.0: Ease of finding skilled employees (1-7)": "find_skill",
    "GCI 4.0: Government ensuring policy stability (1-7)": "policy_stab",
    "GCI 4.0: Competition in professional services (1-7)": "comp_prof",
    "GCI 4.0: Skillset of secondary-education graduates (1-7)": "sec_grad_sk",
    "GCI 4.0: Skillset of university graduates (1-7)": "uni_grad_sk",
    "GCI 4.0: Active labor market policies (1-7)": "labor_polic",
    "GCI 4.0: Government's responsiveness to change (1-7)": "govt_resp_ch",
    "GCI 4.0: Digital skills among active population (1-7)": "digit_skills",
    "GCI 4.0: Future orientation of government (1-7)": "fut_orient",
    "GCI 4.0: Global Competitiveness Index 4.0": "gci_4_index",
    "GCI 4.0: Cost required to start a business (% of GNI per capita)": "cost_start",
    "GCI 4.0: Time required to start a business (days)": "time_start",
    "GCI 4.0: Workers' Rights Index (0-100, best)": "workers_rgt",
    "Total trade in goods and services": "trade_goods",
    "Total trade in goods": "trade_gds",
    "anual_lsci": "anual_lsci",
    "Paved roads %, IRF": "percent_paved",
    "Quality of roads, WEF survey (1-7)": "quality_road_val",
    "Road density Km/GDP (constant 2010 MM US$)": "road_density_gdp",
    "Wastewater Treatment":"wastewater_trt",
    "General government gross debt (Percent of GDP)":"gross_dbt_imf",
    'Economic Complexity Index: Score': 'eci_score',
    'All GHG Agriculture per capita': 'ghg_agro_pc',
    'All GHG Agriculture': 'ghg_agro',
    'All GHG Total per capita': 'ghg_total_pc',
    'All GHG Total': 'ghg_total',
    'All GHG Transportation per capita': 'ghg_trans_pc',
    'All GHG Transportation': 'ghg_trans',
    'Agro GHG as share of total GHG': 'ghg_agro_sh',
    'Trans GHG as share of total GHG': 'ghg_trans_sh',
    'All GHG Agriculture per GDP': 'ghg_agro_pgdp',
    'All GHG Total per GDP': 'ghg_total_pgdp',
    'All GHG Transportation per GDP': 'ghg_trans_pgdp',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Intimate partner or family member: Family member - Sex: Female - Age: Total': 'femic_famil',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Intimate partner or family member: Intimate partner - Sex: Female - Age: Total': 'femic_intim',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Other Perpetrator known to the victim - Sex: Female - Age: Total': 'femic_known',
    'Victims of intentional homicide - Dimension: Total -  Category: Total - Sex: Female - Age: Total': 'femic_total',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Perpetrator unknown to the victim - Sex: Female - Age: Total': 'femic_unkno',
    'Transportation. Gases: CO2. Calculation: Total': 'co2_tr_total',
    'Transportation. Gases: CO2. Calculation: per Capita': 'co2_tr_pc',
    'Transportation. Gases: CO2. Calculation: per GDP': 'co2_tr_pgdp',
    'Total including LUCF. Gases: CO2. Calculation: Total': 'co2_total_em',
    'Industrial Processes. Gases: CO2. Calculation: Total': 'co2_ind_total',
    'Industrial Processes. Gases: CO2. Calculation: per Capita': 'co2_ind_pc',
    'Industrial Processes. Gases: CO2. Calculation: per GDP': 'co2_ind_pgdp',
    'Transportation CO2 as share of total CO2': 'ghg_co2_tr_sh',
    'Industry CO2 as share of total CO2': 'ghg_co2_in_sh',
    'Road density, km/surface area':'road_den_kmsu',
    'eports_pta':'eports_pta',
    'number_fta':'number_fta', 
    "idba_idx":"idba_idx"
    }

In [23]:
variable_names_years = {
    'index_years': 'index',
    'country_years': 'country',
    'country_code_years': 'country_code',
    'year_years': 'year',
    'Central government debt, total (% of GDP)_years': 'gov_debt_gdp',
    'Domestic credit to private sector (% of GDP)_years': 'priv_cred_gdp',
    'Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)_years': 'edu_att_uppr',
    'GDP per person employed (constant 2021 PPP $)_years': 'gdp_per_empl',
    'Intentional homicides (per 100,000 people)_years': 'intent_hom',
    'Interest rate spread (lending rate minus deposit rate, %)_years': 'int_rate_sprd',
    'Internally displaced persons, new displacement associated with disasters (number of cases)_years': 'displ_pax_dis',
    'Logistics performance index: Ability to track and trace consignments, score (1=low to 5=high)_years': 'log_track',
    'Logistics performance index: Competence and quality of logistics services, score (1=low to 5=high)_years': 'log_qual',
    'Logistics performance index: Ease of arranging competitively priced shipments (1=low to 5=high)_years': 'log_ease',
    'Logistics performance index: Efficiency of the clearance process, score (1=low to 5=high)_years': 'log_clear',
    'Logistics performance index: Frequency with which shipments reach consignee within scheduled or expected time, score (1=low to 5=high)_years': 'log_freq',
    'Logistics performance index: Quality of trade and transport-related infrastructure (1=low to 5=high)_years': 'log_infra',
    'People using at least basic drinking water services (% of population)_years': 'basic_drink',
    'People using at least basic sanitation services (% of population)_years': 'basic_san',
    'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)_years': 'pov_hc_215',
    'Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population)_years': 'pov_hc_365',
    'Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population)_years': 'pov_hc_685',
    'Public private partnerships investment in transport (current US$)_years': 'ppp_transp',
    'Renewable energy consumption (% of total final energy consumption)_years': 'renew_enrgy',
    'Research and development expenditure (% of GDP)_years': 'rnd_exp_gdp',
    'Researchers in R&D (per million people)_years': 'resrch_per_m',
    'School enrollment, preprimary (% gross)_years': 'preprim_enrl',
    'School enrollment, primary (% net)_years': 'prim_enrl',
    'School enrollment, secondary (% net)_years': 'sec_enrl',
    'Access to electricity (% of rural population with access_years': 'rural_ele',
    'Access to electricity (% of urban population with access_years': 'urban_ele',
    'Age-standardized suicide rates (per 100 000 population)_years': 'suicide_rate',
    'Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%)_years': 'health_exp',
    'Deaths per 1 000 live births_years': 'deaths_birth',
    'Maternal mortality ratio (per 100 000 live births)_years': 'maternal_mort',
    'Medical doctors (per 10,000)_years': 'med_docs_10k',
    'Premature deaths due to noncommunicable diseases (NCD) as a proportion of all NCD deaths_years': 'prem_deaths',
    'UHC Service Coverage sub-index on noncommunicable diseases_years': 'uhc_ncd_cov',
    'Account, female (% age 15+)_years': 'acct_female',
    'Account, male (% age 15+)_years': 'acct_male',
    'Borrowed from a formal financial institution (% age 15+)_years': 'borrow_fin',
    'Borrowed from a formal financial institution, female (% age 15+)_years': 'borrow_fem',
    'Borrowed from a formal financial institution, male (% age 15+)_years': 'borrow_male',
    'Financial institution account (% age 15+)_years': 'fin_acct',
    'Made or received a digital payment (% age 15+)_years': 'dig_pay',
    'Account, GPI (% age 15+)_years': 'acct_gpi',
    'Borrowed from a formal financial institution, GPI (% age 15+)_years': 'borrow_gpi',
    'Agriculture_years': 'agriculture',
    'Air Pollution_years': 'air_pollut',
    'Air Quality_years': 'air_qual',
    'Biodiversity & Habitat_years': 'bio_hab',
    'Climate Change Mitigation_years': 'clim_chng_m',
    'Fisheries_years': 'fisheries',
    'Forests_years': 'forests',
    'Heavy Metals_years': 'heavy_met',
    'Sanitation & Drinking Water_years': 'san_drink',
    'Waste Management_years': 'waste_mgmt',
    'Water Resources_years': 'water_res',
    'Completion rate, primary education, both sexes (%)_years': 'prim_compl',
    'Completion rate, upper secondary education, both sexes (%)_years': 'upper_sec_c',
    'Completion rate, upper secondary education, GDI (%)_years': 'upper_sec_g',
    'Proportion of students at the end of lower secondary education achieving at least a minimum proficiency level in mathematics, both sexes (%)_years': 'prof_math',
    'Proportion of students at the end of lower secondary education achieving at least a minimum proficiency level in reading, both sexes (%)_years':'prof_read',
    'Net enrolment rate, early childhood education, both sexes (%)_years': 'early_enrl',
    'Percentage of pupils enrolled in primary education who are at least 2 years over-age for their current grade, both sexes (%)_years': 'ovrage_prim',
    'Percentage of pupils enrolled in lower secondary general education who are at least 2 years over-age for their current grade, both sexes (%)_years': 'ovrage_sec',
    'Participation rate of youth and adults in formal and non-formal education and training in the previous 12 months, 15-24 years old, both sexes (%)_years': 'part_formal',
    'Gross enrolment ratio, primary and secondary, female (%)_years': 'gr_enrl_fem',
    'Gross enrolment ratio, primary and secondary, male (%)_years': 'gr_enrl_male',
    'Gross enrolment ratio, primary and secondary, GPI (%)_years': 'gr_enrl_gpi',
    'Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 15+)_years': 'women_viol',
    'Proportion of women subjected to physical and/or sexual violence in the last 12 mon ths (modeled estimate, % of ever partnered women ages 15+)_years': 'viol_12mon',
    'Network Readiness Index: subpillar: Bussineses_years': 'net_read_biz',
    'Population using safely managed sanitation services (%)_years': 'safe_san',
    'Agricultural water withdrawal as % of total renewable water resources_years': 'agri_water',
    'Agricultural R&D spending (constant 2011 PPP$)_years': 'agri_rnd',
    'Agricultural researchers (FTE)_years': 'agri_resrch',
    'Total Agro Emissions_years': 'agro_emiss',
    'rtfpna_years': 'rtfpna',
    'rgdpna_years': 'rgdpna',
    'pop_years': 'pop',
    'population_years': 'population',
    'gdp_ppp_pc_years': 'gdppcppp_const',
    'gdppcppp_const_years': 'gdp_ppp',
    'Electric power transmission and distribution losses (% of output)_years': 'elec_losses',
    'CO2 emissions from transport (% of total fuel combustion)_years': 'co2_trans',
    'Number_of_products_Value_years': 'prod_value',
    'Concentration_Index_Value_years': 'concen_idx',
    'egov_index_years': 'egov_idx',
    'egov_index2_years': 'egov_idx2',
    'pr_pop_rate_years': 'prison_rate',
    'water_use_ef_years': 'water_eff',
    'control_corrupt_years': 'ctrl_corrpt',
    'eci_rank_years': 'eci_rank',
    'Ratio of female to male labor force participation rate (%)_years': 'lf_part_rat',
    ' Percent of children living in monetary poor households in 2022 $2.15_years':'child_p_215',
    ' Percent of children living in monetary poor households in 2022  $3.65_years':'child_pov_365',
    ' Percent of children living in monetary poor households in 2022  $6.85_years':'child_pov_685',
    "GCI 4.0: Distortive effect on competition of taxes and subsidies (1-7)_years": "comp_tax_sub",
    "GCI 4.0: Burden of government regulation (1-7)_years": "govt_reg_bur",
    "GCI 4.0: Extent of market dominance (1-7)_years": "mkt_dominance",
    "GCI 4.0: Hiring and firing practices (1-7)_years": "hire_fire",
    "GCI 4.0: Flexibility of wage determination (1-7)_years": "wage_flex",
    "GCI 4.0: Ease of finding skilled employees (1-7)_years": "find_skill",
    "GCI 4.0: Government ensuring policy stability (1-7)_years": "policy_stab",
    "GCI 4.0: Competition in professional services (1-7)_years": "comp_prof",
    "GCI 4.0: Skillset of secondary-education graduates (1-7)_years": "sec_grad_sk",
    "GCI 4.0: Skillset of university graduates (1-7)_years": "uni_grad_sk",
    "GCI 4.0: Active labor market policies (1-7)_years": "labor_polic",
    "GCI 4.0: Government's responsiveness to change (1-7)_years": "govt_resp_ch",
    "GCI 4.0: Digital skills among active population (1-7)_years": "digit_skills",
    "GCI 4.0: Future orientation of government (1-7)_years": "fut_orient",
    "GCI 4.0: Global Competitiveness Index 4.0_years": "gci_4_index",
    "GCI 4.0: Cost required to start a business (% of GNI per capita)_years": "cost_start",
    "GCI 4.0: Time required to start a business (days)_years": "time_start",
    "GCI 4.0: Workers' Rights Index (0-100, best)_years": "workers_rgt",
    "Total trade in goods and services_years": "trade_goods",
    "Total trade in goods_years": "trade_gds",
    "anual_lsci_years": "anual_lsci",
    "Paved roads %, IRF_years": "percent_paved",
    "Quality of roads, WEF survey (1-7)_years": "quality_road_val",
    "Road density Km/GDP (constant 2010 MM US$)_years": "road_density_gdp",
    "Wastewater Treatment_years":"wastewater_trt",
    "General government gross debt (Percent of GDP)_years":"gross_dbt_imf",
    'Economic Complexity Index: Score_years': 'eci_score',
    'All GHG Agriculture per capita_years': 'ghg_agro_pc',
    'All GHG Agriculture_years': 'ghg_agro',
    'All GHG Total per capita_years': 'ghg_total_pc',
    'All GHG Total_years': 'ghg_total',
    'All GHG Transportation per capita_years': 'ghg_trans_pc',
    'All GHG Transportation_years': 'ghg_trans',
    'Agro GHG as share of total GHG_years': 'ghg_agro_sh',
    'Trans GHG as share of total GHG_years': 'ghg_trans_sh',
    'All GHG Agriculture per GDP_years': 'ghg_agro_pgdp',
    'All GHG Total per GDP_years': 'ghg_total_pgdp',
    'All GHG Transportation per GDP_years': 'ghg_trans_pgdp',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Intimate partner or family member: Family member - Sex: Female - Age: Total_years': 'femic_famil',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Intimate partner or family member: Intimate partner - Sex: Female - Age: Total_years': 'femic_intim',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Other Perpetrator known to the victim - Sex: Female - Age: Total_years': 'femic_known',
    'Victims of intentional homicide - Dimension: Total -  Category: Total - Sex: Female - Age: Total_years': 'femic_total',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Perpetrator unknown to the victim - Sex: Female - Age: Total_years': 'femic_unkno',
    'Transportation. Gases: CO2. Calculation: Total_years': 'co2_tr_total',
    'Transportation. Gases: CO2. Calculation: per Capita_years': 'co2_tr_pc',
    'Transportation. Gases: CO2. Calculation: per GDP_years': 'co2_tr_pgdp',
    'Total including LUCF. Gases: CO2. Calculation: Total_years': 'co2_total_em',
    'Industrial Processes. Gases: CO2. Calculation: Total_years': 'co2_ind_total',
    'Industrial Processes. Gases: CO2. Calculation: per Capita_years': 'co2_ind_pc',
    'Industrial Processes. Gases: CO2. Calculation: per GDP_years': 'co2_ind_pgdp',
    'Transportation CO2 as share of total CO2_years': 'ghg_co2_tr_sh',
    'Industry CO2 as share of total CO2_years': 'ghg_co2_in_sh',
    'Road density, km/surface area_years':'road_den_kmsu',
    'eports_pta_years':'eports_pta',
    'number_fta_years':'number_fta', 
    "idba_idx_years":"idba_idx"
}


In [24]:
variable_names_last = {
    'index_last': 'index',
    'country_last': 'country',
    'country_code_last': 'country_code',
    'year_last': 'year',
    'Central government debt, total (% of GDP)_last': 'gov_debt_gdp',
    'Domestic credit to private sector (% of GDP)_last': 'priv_cred_gdp',
    'Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)_last': 'edu_att_uppr',
    'GDP per person employed (constant 2021 PPP $)_last': 'gdp_per_empl',
    'Intentional homicides (per 100,000 people)_last': 'intent_hom',
    'Interest rate spread (lending rate minus deposit rate, %)_last': 'int_rate_sprd',
    'Internally displaced persons, new displacement associated with disasters (number of cases)_last': 'displ_pax_dis',
    'Logistics performance index: Ability to track and trace consignments, score (1=low to 5=high)_last': 'log_track',
    'Logistics performance index: Competence and quality of logistics services, score (1=low to 5=high)_last': 'log_qual',
    'Logistics performance index: Ease of arranging competitively priced shipments (1=low to 5=high)_last': 'log_ease',
    'Logistics performance index: Efficiency of the clearance process, score (1=low to 5=high)_last': 'log_clear',
    'Logistics performance index: Frequency with which shipments reach consignee within scheduled or expected time, score (1=low to 5=high)_last': 'log_freq',
    'Logistics performance index: Quality of trade and transport-related infrastructure (1=low to 5=high)_last': 'log_infra',
    'People using at least basic drinking water services (% of population)_last': 'basic_drink',
    'People using at least basic sanitation services (% of population)_last': 'basic_san',
    'Poverty headcount ratio at $2.15 a day (2017 PPP) (% of population)_last': 'pov_hc_215',
    'Poverty headcount ratio at $3.65 a day (2017 PPP) (% of population)_last': 'pov_hc_365',
    'Poverty headcount ratio at $6.85 a day (2017 PPP) (% of population)_last': 'pov_hc_685',
    'Public private partnerships investment in transport (current US$)_last': 'ppp_transp',
    'Renewable energy consumption (% of total final energy consumption)_last': 'renew_enrgy',
    'Research and development expenditure (% of GDP)_last': 'rnd_exp_gdp',
    'Researchers in R&D (per million people)_last': 'resrch_per_m',
    'School enrollment, preprimary (% gross)_last': 'preprim_enrl',
    'School enrollment, primary (% net)_last': 'prim_enrl',
    'School enrollment, secondary (% net)_last': 'sec_enrl',
    'Access to electricity (% of rural population with access_last': 'rural_ele',
    'Access to electricity (% of urban population with access_last': 'urban_ele',
    'Age-standardized suicide rates (per 100 000 population)_last': 'suicide_rate',
    'Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%)_last': 'health_exp',
    'Deaths per 1 000 live births_last': 'deaths_birth',
    'Maternal mortality ratio (per 100 000 live births)_last': 'maternal_mort',
    'Medical doctors (per 10,000)_last': 'med_docs_10k',
    'Premature deaths due to noncommunicable diseases (NCD) as a proportion of all NCD deaths_last': 'prem_deaths',
    'UHC Service Coverage sub-index on noncommunicable diseases_last': 'uhc_ncd_cov',
    'Account, female (% age 15+)_last': 'acct_female',
    'Account, male (% age 15+)_last': 'acct_male',
    'Borrowed from a formal financial institution (% age 15+)_last': 'borrow_fin',
    'Borrowed from a formal financial institution, female (% age 15+)_last': 'borrow_fem',
    'Borrowed from a formal financial institution, male (% age 15+)_last': 'borrow_male',
    'Financial institution account (% age 15+)_last': 'fin_acct',
    'Made or received a digital payment (% age 15+)_last': 'dig_pay',
    'Account, GPI (% age 15+)_last': 'acct_gpi',
    'Borrowed from a formal financial institution, GPI (% age 15+)_last': 'borrow_gpi',
    'Agriculture_last': 'agriculture',
    'Air Pollution_last': 'air_pollut',
    'Air Quality_last': 'air_qual',
    'Biodiversity & Habitat_last': 'bio_hab',
    'Climate Change Mitigation_last': 'clim_chng_m',
    'Fisheries_last': 'fisheries',
    'Forests_last': 'forests',
    'Heavy Metals_last': 'heavy_met',
    'Sanitation & Drinking Water_last': 'san_drink',
    'Waste Management_last': 'waste_mgmt',
    'Water Resources_last': 'water_res',
    'Completion rate, primary education, both sexes (%)_last': 'prim_compl',
    'Completion rate, upper secondary education, both sexes (%)_last': 'upper_sec_c',
    'Completion rate, upper secondary education, GDI (%)_last': 'upper_sec_g',
    'Proportion of students at the end of lower secondary education achieving at least a minimum proficiency level in mathematics, both sexes (%)_last': 'prof_math',
    'Proportion of students at the end of lower secondary education achieving at least a minimum proficiency level in reading, both sexes (%)_last':'prof_read',
    'Net enrolment rate, early childhood education, both sexes (%)_last': 'early_enrl',
    'Percentage of pupils enrolled in primary education who are at least 2 years over-age for their current grade, both sexes (%)_last': 'ovrage_prim',
    'Percentage of pupils enrolled in lower secondary general education who are at least 2 years over-age for their current grade, both sexes (%)_last': 'ovrage_sec',
    'Participation rate of youth and adults in formal and non-formal education and training in the previous 12 months, 15-24 years old, both sexes (%)_last': 'part_formal',
    'Gross enrolment ratio, primary and secondary, female (%)_last': 'gr_enrl_fem',
    'Gross enrolment ratio, primary and secondary, male (%)_last': 'gr_enrl_male',
    'Gross enrolment ratio, primary and secondary, GPI (%)_last': 'gr_enrl_gpi',
    'Proportion of women who have ever experienced intimate partner violence (modeled estimate, % of ever partnered women ages 15+)_last': 'women_viol',
    'Proportion of women subjected to physical and/or sexual violence in the last 12 mon ths (modeled estimate, % of ever partnered women ages 15+)_last': 'viol_12mon',
    'Network Readiness Index: subpillar: Bussineses_last': 'net_read_biz',
    'Population using safely managed sanitation services (%)_last': 'safe_san',
    'Agricultural water withdrawal as % of total renewable water resources_last': 'agri_water',
    'Agricultural R&D spending (constant 2011 PPP$)_last': 'agri_rnd',
    'Agricultural researchers (FTE)_last': 'agri_resrch',
    'Total Agro Emissions_last': 'agro_emiss',
    'rtfpna_last': 'rtfpna',
    'rgdpna_last': 'rgdpna',
    'pop_last': 'pop',
    'population_last': 'population',
    'gdp_ppp_pc_last': 'gdppcppp_const',
    'gdppcppp_const_last': 'gdp_ppp',
    'Electric power transmission and distribution losses (% of output)_last': 'elec_losses',
    'CO2 emissions from transport (% of total fuel combustion)_last': 'co2_trans',
    'Number_of_products_Value_last': 'prod_value',
    'Concentration_Index_Value_last': 'concen_idx',
    'egov_index_last': 'egov_idx',
    'egov_index2_last': 'egov_idx2',
    'pr_pop_rate_last': 'prison_rate',
    'water_use_ef_last': 'water_eff',
    'control_corrupt_last': 'ctrl_corrpt',
    'eci_rank_last': 'eci_rank',
    'Ratio of female to male labor force participation rate (%)_last': 'lf_part_rat',
    ' Percent of children living in monetary poor households in 2022 $2.15_last':'child_p_215',
    ' Percent of children living in monetary poor households in 2022  $3.65_last':'child_pov_365',
    ' Percent of children living in monetary poor households in 2022  $6.85_last':'child_pov_685',
    "GCI 4.0: Distortive effect on competition of taxes and subsidies (1-7)_last": "comp_tax_sub",
    "GCI 4.0: Burden of government regulation (1-7)_last": "govt_reg_bur",
    "GCI 4.0: Extent of market dominance (1-7)_last": "mkt_dominance",
    "GCI 4.0: Hiring and firing practices (1-7)_last": "hire_fire",
    "GCI 4.0: Flexibility of wage determination (1-7)_last": "wage_flex",
    "GCI 4.0: Ease of finding skilled employees (1-7)_last": "find_skill",
    "GCI 4.0: Government ensuring policy stability (1-7)_last": "policy_stab",
    "GCI 4.0: Competition in professional services (1-7)_last": "comp_prof",
    "GCI 4.0: Skillset of secondary-education graduates (1-7)_last": "sec_grad_sk",
    "GCI 4.0: Skillset of university graduates (1-7)_last": "uni_grad_sk",
    "GCI 4.0: Active labor market policies (1-7)_last": "labor_polic",
    "GCI 4.0: Government's responsiveness to change (1-7)_last": "govt_resp_ch",
    "GCI 4.0: Digital skills among active population (1-7)_last": "digit_skills",
    "GCI 4.0: Future orientation of government (1-7)_last": "fut_orient",
    "GCI 4.0: Global Competitiveness Index 4.0_last": "gci_4_index",
    "GCI 4.0: Cost required to start a business (% of GNI per capita)_last": "cost_start",
    "GCI 4.0: Time required to start a business (days)_last": "time_start",
    "GCI 4.0: Workers' Rights Index (0-100, best)_last": "workers_rgt",
    "Total trade in goods and services_last": "trade_goods",
    "Total trade in goods_last": "trade_gds",
    "anual_lsci_last": "anual_lsci",
    "Paved roads %, IRF_last": "percent_paved",
    "Quality of roads, WEF survey (1-7)_last": "quality_road_val",
    "Road density Km/GDP (constant 2010 MM US$)_last": "road_density_gdp",
    "Wastewater Treatment_last":"wastewater_trt",
    "General government gross debt (Percent of GDP)_last":"gross_dbt_imf",
    'Economic Complexity Index: Score_last': 'eci_score',
    'All GHG Agriculture per capita_last': 'ghg_agro_pc',
    'All GHG Agriculture_last': 'ghg_agro',
    'All GHG Total per capita_last': 'ghg_total_pc',
    'All GHG Total_last': 'ghg_total',
    'All GHG Transportation per capita_last': 'ghg_trans_pc',
    'All GHG Transportation_last': 'ghg_trans',
    'Agro GHG as share of total GHG_last': 'ghg_agro_sh',
    'Trans GHG as share of total GHG_last': 'ghg_trans_sh',
    'All GHG Agriculture per GDP_last': 'ghg_agro_pgdp',
    'All GHG Total per GDP_last': 'ghg_total_pgdp',
    'All GHG Transportation per GDP_last': 'ghg_trans_pgdp',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Intimate partner or family member: Family member - Sex: Female - Age: Total_last': 'femic_famil',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Intimate partner or family member: Intimate partner - Sex: Female - Age: Total_last': 'femic_intim',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Other Perpetrator known to the victim - Sex: Female - Age: Total_last': 'femic_known',
    'Victims of intentional homicide - Dimension: Total -  Category: Total - Sex: Female - Age: Total_last': 'femic_total',
    'Victims of intentional homicide - Dimension: by relationship to perpetrator -  Category: Perpetrator unknown to the victim - Sex: Female - Age: Total_last': 'femic_unkno',
    'Transportation. Gases: CO2. Calculation: Total_last': 'co2_tr_total',
    'Transportation. Gases: CO2. Calculation: per Capita_last': 'co2_tr_pc',
    'Transportation. Gases: CO2. Calculation: per GDP_last': 'co2_tr_pgdp',
    'Total including LUCF. Gases: CO2. Calculation: Total_last': 'co2_total_em',
    'Industrial Processes. Gases: CO2. Calculation: Total_last': 'co2_ind_total',
    'Industrial Processes. Gases: CO2. Calculation: per Capita_last': 'co2_ind_pc',
    'Industrial Processes. Gases: CO2. Calculation: per GDP_last': 'co2_ind_pgdp',
    'Transportation CO2 as share of total CO2_last': 'ghg_co2_tr_sh',
    'Industry CO2 as share of total CO2_last': 'ghg_co2_in_sh',
    'Road density, km/surface area_last':'road_den_kmsu',
    'eports_pta_last':'eports_pta',
    'number_fta_last':'number_fta', 
    "idba_idx_last":"idba_idx"
}


In [25]:
main_cs = main_cs.rename(columns = variable_names)
main_cs_years = main_cs_years.rename(columns = variable_names_years)
main_cs_last = main_cs_last.rename(columns = variable_names)
main_cs_years_last = main_cs_years_last.rename(columns = variable_names_last)

In [26]:
# Define the columns you want to move to the front
columns_to_move = ['country_code', 'country', 'year']
# Create a new column list with the specified columns first, followed by the remaining columns
new_column_order = columns_to_move + [col for col in main_cs.columns if col not in columns_to_move]
main_cs = main_cs[new_column_order]
del columns_to_move

In [27]:
main_cs.to_csv("../data/output/main_cs11.csv")
main_cs_last.to_csv("../data/output/main_cs11_last.csv")
main_cs_years_last.to_csv("../data/output/main_cs11_years_last.csv")

In [28]:
column_list_string = ' '.join(main_cs.columns)
print(column_list_string)

country_code country year Unnamed: 0.9 Unnamed: 0.8 Unnamed: 0.7 Unnamed: 0.6 Unnamed: 0.5 Unnamed: 0.4 Unnamed: 0.3 Unnamed: 0.2 Unnamed: 0 Unnamed: 0.1 Unnamed: 0_x index gov_debt_gdp priv_cred_gdp edu_att_uppr gdp_per_empl intent_hom int_rate_sprd displ_pax_dis log_track log_qual log_ease log_clear log_freq log_infra basic_drink basic_san pov_hc_215 pov_hc_365 pov_hc_685 ppp_transp renew_enrgy rnd_exp_gdp resrch_per_m preprim_enrl prim_enrl sec_enrl rural_ele urban_ele suicide_rate health_exp deaths_birth maternal_mort med_docs_10k prem_deaths uhc_ncd_cov acct_female acct_male borrow_fin borrow_fem borrow_male fin_acct dig_pay acct_gpi borrow_gpi agriculture air_pollut air_qual bio_hab clim_chng_m fisheries forests heavy_met san_drink waste_mgmt water_res prim_compl upper_sec_c upper_sec_g prof_math prof_read early_enrl ovrage_prim ovrage_sec part_formal gr_enrl_fem gr_enrl_male gr_enrl_gpi women_viol viol_12mon net_read_biz safe_san agri_water agri_rnd agri_resrch agro_emiss rtfpna

country checks and country trimming

In [29]:
#ctr_label
all_countries = [
    "ARG", "BHS", "BRB", "BRA", "CHL", "MEX", "TTO", "URY", "VEN",
    "BLZ", "BOL", "COL", "CRI", "ECU", "SLV", "GTM", "GUY", "HTI",
    "HND", "JAM", "NIC", "PAN", "PRY", "PER", "DOM", "SUR"
]

# Create the new column with values from all_countries list where the ISO codes match
main_cs['ctr_label'] = main_cs['country_code'].apply(lambda x: x if x in all_countries else None)


In [30]:
main_cs = main_cs.loc[main_cs["population"] > 0.5] #164

In [31]:
#142

# List of ISO codes to be removed
iso_codes_to_remove = ['LBY', 'IRQ', 'COG', 'AGO', 'KWT', 'SAU', 'OMN', 'GUY', 'AZE', 'IRN', 'TCD', 'ARE', 'GAB', 'QAT', 'GNQ', 'KAZ', 'DZA', 'VEN']

# Removing the specified countries from the DataFrame
main_cs = main_cs[~main_cs['country_code'].isin(iso_codes_to_remove)]

# List of ISO codes for tax havens to be removed
tax_havens_iso3 = [
    "AND",  # Andorra 
    "BMU",  # Bermuda
    "VGB",  # British Virgin Islands
    "CYM",  # Cayman Islands
    "GGY",  # Guernsey
    "HKG",  # Hong Kong
    "IMN",  # Isle of Man
    "JEY",  # Jersey
    "LUX",  # Luxembourg
    "MLT",  # Malta
    "MCO",  # Monaco 
    "SMR",  # San Marino
    "SYC",  # Seychelles
    "SGP",  # Singapore
    "CHE",  # Switzerland
    "LIE",  # Liechtenstein
    "MHL",  # Marshall Islands
]

# Removing the specified tax havens from the DataFrame, if they exist
main_cs = main_cs[~main_cs['country_code'].isin(tax_havens_iso3)]

In [32]:
columns_array = main_cs.columns.to_list()
print(columns_array)
main_cs = main_cs[['country_code', 'country', 'year', 'gov_debt_gdp', 'priv_cred_gdp', 'edu_att_uppr', 'gdp_per_empl', 'intent_hom', 'int_rate_sprd', 'displ_pax_dis', 'log_track', 'log_qual', 'log_ease', 'log_clear', 'log_freq', 'log_infra', 'basic_drink', 'basic_san', 'pov_hc_215', 'pov_hc_365', 'pov_hc_685', 'ppp_transp', 'renew_enrgy', 'rnd_exp_gdp', 'resrch_per_m', 'preprim_enrl', 'prim_enrl', 'sec_enrl', 'rural_ele', 'urban_ele', 'suicide_rate', 'health_exp', 'deaths_birth', 'maternal_mort', 'med_docs_10k', 'prem_deaths', 'uhc_ncd_cov', 'acct_female', 'acct_male', 'borrow_fin', 'borrow_fem', 'borrow_male', 'fin_acct', 'dig_pay', 'acct_gpi', 'borrow_gpi', 'agriculture', 'air_pollut', 'air_qual', 'bio_hab', 'clim_chng_m', 'fisheries', 'forests', 'heavy_met', 'san_drink', 'waste_mgmt', 'water_res', 'prim_compl', 'upper_sec_c', 'upper_sec_g', 'prof_math', 'prof_read', 'early_enrl', 'ovrage_prim', 'ovrage_sec', 'part_formal', 'gr_enrl_fem', 'gr_enrl_male', 'gr_enrl_gpi', 'women_viol', 'viol_12mon', 'net_read_biz', 'safe_san', 'agri_water', 'agri_rnd', 'agri_resrch', 'agro_emiss', 'rtfpna', 'rgdpna', 'pop', 'gdp_ppp', 'elec_losses', 'co2_trans', 'prod_value', 'concen_idx', 'egov_idx', 'prison_rate', 'water_eff', 'ctrl_corrpt', 'eci_rank', 'lf_part_rat', 'child_p_215', 'child_pov_365', 'child_pov_685', 'population', 'gdppcppp_const', 'trade_gds', 'trade_goods', 'anual_lsci', 'labor_polic', 'govt_reg_bur', 'comp_prof', 'cost_start', 'digit_skills', 'comp_tax_sub', 'find_skill', 'mkt_dominance', 'wage_flex', 'fut_orient', 'gci_4_index', 'policy_stab', 'govt_resp_ch', 'hire_fire', 'sec_grad_sk', 'uni_grad_sk', 'time_start', 'workers_rgt', 'ctr_label', "percent_paved", "quality_road_val", "road_density_gdp", "wastewater_trt", "gross_dbt_imf", "eci_score", "ghg_agro_pc", "ghg_agro", "ghg_total_pc", "ghg_total", "ghg_trans_pc", "ghg_trans","ghg_agro_sh","ghg_trans_sh", "ghg_agro_pgdp", "ghg_total_pgdp", "ghg_trans_pgdp", "femic_famil", "femic_intim", "femic_known", "femic_total", "femic_unkno", 'co2_tr_total','co2_tr_pc','co2_tr_pgdp','co2_total_em','co2_ind_total','co2_ind_pc','co2_ind_pgdp','ghg_co2_tr_sh','ghg_co2_in_sh', 'road_den_kmsu', "eports_pta", "number_fta", "egov_idx2", "idba_idx"]]

['country_code', 'country', 'year', 'Unnamed: 0.9', 'Unnamed: 0.8', 'Unnamed: 0.7', 'Unnamed: 0.6', 'Unnamed: 0.5', 'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0_x', 'index', 'gov_debt_gdp', 'priv_cred_gdp', 'edu_att_uppr', 'gdp_per_empl', 'intent_hom', 'int_rate_sprd', 'displ_pax_dis', 'log_track', 'log_qual', 'log_ease', 'log_clear', 'log_freq', 'log_infra', 'basic_drink', 'basic_san', 'pov_hc_215', 'pov_hc_365', 'pov_hc_685', 'ppp_transp', 'renew_enrgy', 'rnd_exp_gdp', 'resrch_per_m', 'preprim_enrl', 'prim_enrl', 'sec_enrl', 'rural_ele', 'urban_ele', 'suicide_rate', 'health_exp', 'deaths_birth', 'maternal_mort', 'med_docs_10k', 'prem_deaths', 'uhc_ncd_cov', 'acct_female', 'acct_male', 'borrow_fin', 'borrow_fem', 'borrow_male', 'fin_acct', 'dig_pay', 'acct_gpi', 'borrow_gpi', 'agriculture', 'air_pollut', 'air_qual', 'bio_hab', 'clim_chng_m', 'fisheries', 'forests', 'heavy_met', 'san_drink', 'waste_mgmt', 'water_res', 'prim_compl', 'upper_se

In [33]:
import numpy as np 
import pandas as pd

# Convert the 'acct_gpi' column to a numeric type, forcing non-numeric values to NaN
main_cs['acct_gpi'] = pd.to_numeric(main_cs['acct_gpi'], errors='coerce') 
main_cs.loc[main_cs["country_code"]=="TKM","acct_gpi"] = np.nan


# List of columns to round
columns_to_round = [
    'ghg_agro_pgdp', 'ghg_total_pgdp', 'ghg_trans_pgdp', 'ghg_agro_sh',
    'ghg_trans_sh', 'femic_famil', 'femic_intim', 'femic_known',
    'femic_total', 'femic_unkno'
]

# Round the specified columns to 4 decimal places
for column in columns_to_round:
    main_cs[column] = main_cs[column].round(8)


# Drop one instance of each duplicated column
main_cs = main_cs.loc[:, ~main_cs.columns.duplicated()]
 
 
# Continue with saving the data or any other operation
main_cs.to_stata("../data/output/main_cs11.dta")

In [34]:
main_cs.to_csv("../data/output/main_cs11.csv")

##### **Aux, for Stata**

In [35]:
import pandas as pd
clasificacion_variables = pd.read_excel("../data/output/clasificacion_variables.xlsx")
clasificacion_variables.to_stata("../data/output/clasificacion_variables.dta")

## FOR YEARS USED PER COUNTRY!

In [36]:
main_cs_years = main_cs_years[['country_code', 'country', 'year', 'gov_debt_gdp', 'priv_cred_gdp', 'edu_att_uppr', 'gdp_per_empl', 'intent_hom', 'int_rate_sprd', 'displ_pax_dis', 'log_track', 'log_qual', 'log_ease', 'log_clear', 'log_freq', 'log_infra', 'basic_drink', 'basic_san', 'pov_hc_215', 'pov_hc_365', 'pov_hc_685', 'ppp_transp', 'renew_enrgy', 'rnd_exp_gdp', 'resrch_per_m', 'preprim_enrl', 'prim_enrl', 'sec_enrl', 'rural_ele', 'urban_ele', 'suicide_rate', 'health_exp', 'deaths_birth', 'maternal_mort', 'med_docs_10k', 'prem_deaths', 'uhc_ncd_cov', 'acct_female', 'acct_male', 'borrow_fin', 'borrow_fem', 'borrow_male', 'fin_acct', 'dig_pay', 'acct_gpi', 'borrow_gpi', 'agriculture', 'air_pollut', 'air_qual', 'bio_hab', 'clim_chng_m', 'fisheries', 'forests', 'heavy_met', 'san_drink', 'waste_mgmt', 'water_res', 'prim_compl', 'upper_sec_c', 'upper_sec_g', 'prof_math', 'prof_read', 'early_enrl', 'ovrage_prim', 'ovrage_sec', 'part_formal', 'gr_enrl_fem', 'gr_enrl_male', 'gr_enrl_gpi', 'women_viol', 'viol_12mon', 'net_read_biz', 'safe_san', 'agri_water', 'agri_rnd', 'agri_resrch', 'agro_emiss', 'rtfpna', 'rgdpna', 'pop', 'gdp_ppp', 'elec_losses', 'co2_trans', 'prod_value', 'concen_idx', 'egov_idx', 'prison_rate', 'water_eff', 'ctrl_corrpt', 'eci_rank', 'lf_part_rat', 'child_p_215', 'child_pov_365', 'child_pov_685', 'population', 'gdppcppp_const', 'trade_gds', 'trade_goods', 'anual_lsci', 'labor_polic', 'govt_reg_bur', 'comp_prof', 'cost_start', 'digit_skills', 'comp_tax_sub', 'find_skill', 'mkt_dominance', 'wage_flex', 'fut_orient', 'gci_4_index', 'policy_stab', 'govt_resp_ch', 'hire_fire', 'sec_grad_sk', 'uni_grad_sk', 'time_start', 'workers_rgt', "percent_paved", "quality_road_val", "road_density_gdp", "wastewater_trt", "gross_dbt_imf", "eci_score", "ghg_agro_pc", "ghg_agro", "ghg_total_pc", "ghg_total", "ghg_trans_pc", "ghg_trans","ghg_agro_sh","ghg_trans_sh", "ghg_agro_pgdp", "ghg_total_pgdp", "ghg_trans_pgdp", "femic_famil", "femic_intim", "femic_known", "femic_total", "femic_unkno", 'co2_tr_total','co2_tr_pc','co2_tr_pgdp','co2_total_em','co2_ind_total','co2_ind_pc','co2_ind_pgdp','ghg_co2_tr_sh','ghg_co2_in_sh','road_den_kmsu', "eports_pta", "number_fta" , "egov_idx2", "idba_idx" ]]

main_cs_last = main_cs_last[['country_code', 'country', 'year', 'gov_debt_gdp', 'priv_cred_gdp', 'edu_att_uppr', 'gdp_per_empl', 'intent_hom', 'int_rate_sprd', 'displ_pax_dis', 'log_track', 'log_qual', 'log_ease', 'log_clear', 'log_freq', 'log_infra', 'basic_drink', 'basic_san', 'pov_hc_215', 'pov_hc_365', 'pov_hc_685', 'ppp_transp', 'renew_enrgy', 'rnd_exp_gdp', 'resrch_per_m', 'preprim_enrl', 'prim_enrl', 'sec_enrl', 'rural_ele', 'urban_ele', 'suicide_rate', 'health_exp', 'deaths_birth', 'maternal_mort', 'med_docs_10k', 'prem_deaths', 'uhc_ncd_cov', 'acct_female', 'acct_male', 'borrow_fin', 'borrow_fem', 'borrow_male', 'fin_acct', 'dig_pay', 'acct_gpi', 'borrow_gpi', 'agriculture', 'air_pollut', 'air_qual', 'bio_hab', 'clim_chng_m', 'fisheries', 'forests', 'heavy_met', 'san_drink', 'waste_mgmt', 'water_res', 'prim_compl', 'upper_sec_c', 'upper_sec_g', 'prof_math', 'prof_read', 'early_enrl', 'ovrage_prim', 'ovrage_sec', 'part_formal', 'gr_enrl_fem', 'gr_enrl_male', 'gr_enrl_gpi', 'women_viol', 'viol_12mon', 'net_read_biz', 'safe_san', 'agri_water', 'agri_rnd', 'agri_resrch', 'agro_emiss', 'rtfpna', 'rgdpna', 'pop', 'gdp_ppp', 'elec_losses', 'co2_trans', 'prod_value', 'concen_idx', 'egov_idx', 'prison_rate', 'water_eff', 'ctrl_corrpt', 'eci_rank', 'lf_part_rat', 'child_p_215', 'child_pov_365', 'child_pov_685', 'population', 'gdppcppp_const', 'trade_gds', 'trade_goods', 'anual_lsci', 'labor_polic', 'govt_reg_bur', 'comp_prof', 'cost_start', 'digit_skills', 'comp_tax_sub', 'find_skill', 'mkt_dominance', 'wage_flex', 'fut_orient', 'gci_4_index', 'policy_stab', 'govt_resp_ch', 'hire_fire', 'sec_grad_sk', 'uni_grad_sk', 'time_start', 'workers_rgt', "percent_paved", "quality_road_val", "road_density_gdp", "wastewater_trt", "gross_dbt_imf", "eci_score", "ghg_agro_pc", "ghg_agro", "ghg_total_pc", "ghg_total", "ghg_trans_pc", "ghg_trans","ghg_agro_sh","ghg_trans_sh", "ghg_agro_pgdp", "ghg_total_pgdp", "ghg_trans_pgdp", "femic_famil", "femic_intim", "femic_known", "femic_total", "femic_unkno", 'co2_tr_total','co2_tr_pc','co2_tr_pgdp','co2_total_em','co2_ind_total','co2_ind_pc','co2_ind_pgdp','ghg_co2_tr_sh','ghg_co2_in_sh','road_den_kmsu', "eports_pta", "number_fta", "egov_idx2", "idba_idx"  ]]

main_cs_years_last = main_cs_years_last[['country_code', 'country', 'year', 'gov_debt_gdp', 'priv_cred_gdp', 'edu_att_uppr', 'gdp_per_empl', 'intent_hom', 'int_rate_sprd', 'displ_pax_dis', 'log_track', 'log_qual', 'log_ease', 'log_clear', 'log_freq', 'log_infra', 'basic_drink', 'basic_san', 'pov_hc_215', 'pov_hc_365', 'pov_hc_685', 'ppp_transp', 'renew_enrgy', 'rnd_exp_gdp', 'resrch_per_m', 'preprim_enrl', 'prim_enrl', 'sec_enrl', 'rural_ele', 'urban_ele', 'suicide_rate', 'health_exp', 'deaths_birth', 'maternal_mort', 'med_docs_10k', 'prem_deaths', 'uhc_ncd_cov', 'acct_female', 'acct_male', 'borrow_fin', 'borrow_fem', 'borrow_male', 'fin_acct', 'dig_pay', 'acct_gpi', 'borrow_gpi', 'agriculture', 'air_pollut', 'air_qual', 'bio_hab', 'clim_chng_m', 'fisheries', 'forests', 'heavy_met', 'san_drink', 'waste_mgmt', 'water_res', 'prim_compl', 'upper_sec_c', 'upper_sec_g', 'prof_math', 'prof_read', 'early_enrl', 'ovrage_prim', 'ovrage_sec', 'part_formal', 'gr_enrl_fem', 'gr_enrl_male', 'gr_enrl_gpi', 'women_viol', 'viol_12mon', 'net_read_biz', 'safe_san', 'agri_water', 'agri_rnd', 'agri_resrch', 'agro_emiss', 'rtfpna', 'rgdpna', 'pop', 'gdp_ppp', 'elec_losses', 'co2_trans', 'prod_value', 'concen_idx', 'egov_idx', 'prison_rate', 'water_eff', 'ctrl_corrpt', 'eci_rank', 'lf_part_rat', 'child_p_215', 'child_pov_365', 'child_pov_685', 'population', 'gdppcppp_const', 'trade_gds', 'trade_goods', 'anual_lsci', 'labor_polic', 'govt_reg_bur', 'comp_prof', 'cost_start', 'digit_skills', 'comp_tax_sub', 'find_skill', 'mkt_dominance', 'wage_flex', 'fut_orient', 'gci_4_index', 'policy_stab', 'govt_resp_ch', 'hire_fire', 'sec_grad_sk', 'uni_grad_sk', 'time_start', 'workers_rgt', "percent_paved", "quality_road_val", "road_density_gdp", "wastewater_trt", "gross_dbt_imf", "eci_score", "ghg_agro_pc", "ghg_agro", "ghg_total_pc", "ghg_total", "ghg_trans_pc", "ghg_trans","ghg_agro_sh","ghg_trans_sh", "ghg_agro_pgdp", "ghg_total_pgdp", "ghg_trans_pgdp", "femic_famil", "femic_intim", "femic_known", "femic_total", "femic_unkno", 'co2_tr_total','co2_tr_pc','co2_tr_pgdp','co2_total_em','co2_ind_total','co2_ind_pc','co2_ind_pgdp','ghg_co2_tr_sh','ghg_co2_in_sh','road_den_kmsu', "eports_pta", "number_fta", "egov_idx2", "idba_idx" ]]

In [37]:
main_cs_years = main_cs_years.loc[main_cs_years["country"]=="Uruguay",]
main_cs_years.to_csv("../data/output/main_cs_years_uy.csv")

In [38]:
lac = [
    "ARG", "BHS", "BRB", "BRA", "CHL", "MEX", "TTO", "URY", "VEN",
    "BLZ", "BOL", "COL", "CRI", "ECU", "SLV", "GTM", "GUY", "HTI",
    "HND", "JAM", "NIC", "PAN", "PRY", "PER", "DOM", "SUR"
]

oecd = [
    "AUS", "AUT", "BEL", "CAN", "CZE", "DNK", "EST", 
    "FIN", "FRA", "DEU", "GRC", "HUN", "ISL", "IRL", "ISR", "ITA", "JPN", 
    "KOR", "LVA", "LTU", "LUX", "NLD", "NZL", "NOR", "POL", "PRT", 
    "SVK", "SVN", "ESP", "SWE", "CHE", "TUR", "GBR", "USA"
]

# Filter and save main_cs_last for countries in the LAC region
main_cs_last_lac = main_cs_last.loc[main_cs_last["country_code"].isin(lac),]
main_cs_last_lac.to_csv("../data/output/main_cs_last_lac.csv", index=False)

# Filter and save main_cs_years_last for countries in the LAC region
main_cs_years_last_lac = main_cs_years_last.loc[main_cs_years_last["country_code"].isin(lac),]
main_cs_years_last_lac.to_csv("../data/output/main_cs_years_last_lac.csv", index=False)

# Filter and save main_cs_last for countries in the OECD region
main_cs_last_oecd = main_cs_last.loc[main_cs_last["country_code"].isin(oecd),]
main_cs_last_oecd.to_csv("../data/output/main_cs_last_oecd.csv", index=False)

# Filter and save main_cs_years_last for countries in the OECD region
main_cs_years_last_oecd = main_cs_years_last.loc[main_cs_years_last["country_code"].isin(oecd),]
main_cs_years_last_oecd.to_csv("../data/output/main_cs_years_last_oecd.csv", index=False)

main_cs_last_uy = main_cs_last.loc[main_cs_last["country"]=="Uruguay",]
main_cs_last_uy.to_csv("../data/output/main_cs_last_uy.csv")
main_cs_years_last_uy = main_cs_years_last.loc[main_cs_years_last["country"]=="Uruguay",]
main_cs_years_last_uy.to_csv("../data/output/main_cs_years_last_uy.csv")

